In [1]:
import requests
import json
import sys
import os
sys.path.insert(0, '../ghtesting')

import pandas as pd
from ghdatabase import GHDatabase
from ghrepo import GHRepo
from tqdm.auto import tqdm
from datetime import datetime
from git import Repo
import pickle

In [2]:
repo_db = GHDatabase('ecs260', 'webframework_repos', os.environ['CONNECTION_STRING'])

repos = list(repo_db.get_repos())

num_repos = len(repos)
print(f'Number of repos: {num_repos}')

Number of repos: 7644


In [3]:
codecov_db = GHDatabase('ecs260', 'codecov_api', os.environ['CONNECTION_STRING'])
codecov_api = list(codecov_db.get_repos())

# Get initial responses from codecov api

In [4]:
def get_codecov_response(_id, page=None):
    page = f'?page=%s&limit=250' % page if page is not None else ''
    url = 'http://codecov.io/api/gh/%s/commits%s' % (_id, page)
    r = requests.get(url)
    return r.json()

In [5]:
ids_done = set(map(lambda x: x['_id'], codecov_api))
remaining = [repo for repo in repos if repo['_id'] not in ids_done]

for repo in tqdm(remaining):
    
    # get data from api
    _id = repo['_id']
    page = 1
    first_response = None
    repo_commits = []
    
    while True:
        
        # get commits page from api
        response = get_codecov_response(_id, page)
        
        # save first response
        if page == 1:
            first_response = response
            
        # get commits if available
        commits = response.get('commits', [])
        
        # extend repo commits collection
        repo_commits.extend(commits)
        
        # increment page number
        page += 1
        
        # break if no new commits in current page
        if len(commits) == 0:
            break
       
    codecov_db.update_repo(dict(
        _id=_id,
        data=first_response,
        commits=repo_commits
    ))

0it [00:00, ?it/s]

# Check coverage

In [6]:
active = []
inactive = []
errors = []

for response in codecov_api:
    if 'error' in response['data']:
        errors.append(response)
        
    elif response['data']['repo']['active']:
        active.append(response)
    
    elif not response['data']['repo']['active']:
        inactive.append(response)

In [7]:
print('Errors', len(errors))
print('Inactive', len(inactive))
print('Active', len(active))

Errors 1335
Inactive 5546
Active 763
